<a href="https://colab.research.google.com/github/PUBPOL-2130/notebooks/blob/main/helper-notebooks/Week6-supplemental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Week 6: Practice Prorating Data



First, we'll load the data from the Week6 notebook and install the same packages (such as maup).

In [1]:
!curl -OL https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week6_NY_precincts.zip
!unzip week6_NY_precincts.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 21.9M  100 21.9M    0     0  8864k      0  0:00:02  0:00:02 --:--:-- 12.1M
Archive:  week6_NY_precincts.zip
   creating: week6_NY_precincts/
  inflating: __MACOSX/._week6_NY_precincts  
  inflating: week6_NY_precincts/week6-NY-precincts.dbf  
  inflating: __MACOSX/week6_NY_precincts/._week6-NY-precincts.dbf  
  inflating: week6_NY_precincts/week6-NY-precincts.cpg  
  inflating: __MACOSX/week6_NY_precincts/._week6-NY-precincts.cpg  
  inflating: week6_NY_precincts/week6-NY-precincts.shp  
  inflating: __MACOSX/week6_NY_precincts/._week6-NY-precincts.shp  
  inflating: week6_NY_precincts/week6-NY-precincts.shx  
  inflating: __MACOSX/week6_NY_precincts/._week6-NY-precincts.shx  
  inflating: week6_NY_precincts/week6-NY-precincts.prj  
  inflating: __M

In [2]:
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_block_gdf.zip"
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_block_with_race_gdf.zip"
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_block_with_race_populated_gdf.zip"
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_tract_gdf.zip"
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_tract_with_acs_gdf.zip"


!unzip week4_36_county_block_gdf.zip
!unzip week4_36_county_block_with_race_gdf.zip
!unzip week4_36_county_block_with_race_populated_gdf.zip
!unzip week4_36_county_tract_gdf.zip
!unzip week4_36_county_tract_with_acs_gdf.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1539k  100 1539k    0     0   872k      0  0:00:01  0:00:01 --:--:-- 7389k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2176k  100 2176k    0     0  1181k      0  0:00:01  0:00:01 --:--:-- 2092k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1678k  100 1678k    0     0  1140k      0  0:00:01  0:00:01 --:--:-- 2712k
  % Total    % Received % Xferd  Average Speed   Tim

In [1]:
!pip install maup

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import maup
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_formats = ["retina"]

In [3]:
state_fips = "36"    # New York
county_fips = "047"
county_name = "Kings County"  # Also known as Brooklyn

In [4]:
county_block_gdf = gpd.read_file(f"week4_{state_fips}_county_block_gdf")
county_block_with_race_gdf = gpd.read_file(f"week4_{state_fips}_county_block_with_race_gdf")
county_block_with_race_populated_gdf = gpd.read_file(f"week4_{state_fips}_county_block_with_race_populated_gdf")
county_tract_gdf = gpd.read_file(f"week4_{state_fips}_county_tract_gdf")
county_tract_with_acs_gdf = gpd.read_file(f"week4_{state_fips}_county_tract_with_acs_gdf")

In [5]:
county_block_gdf.set_index("GEOID20", inplace=True)
county_block_gdf.index = county_block_gdf.index.astype(str)
county_block_with_race_gdf.set_index("GEOID20", inplace=True)
county_block_with_race_gdf.index = county_block_with_race_gdf.index.astype(str)
county_block_with_race_populated_gdf.set_index("GEOID20", inplace=True)
county_block_with_race_gdf.index = county_block_with_race_gdf.index.astype(str)
county_tract_gdf.set_index("GEOID", inplace=True)
county_tract_gdf.index = county_tract_gdf.index.astype(str)
county_tract_with_acs_gdf.set_index("GEOID", inplace=True)
county_tract_with_acs_gdf.index = county_tract_with_acs_gdf.index.astype(str)

In [6]:
choropleth_style = dict(
    edgecolor="0.1",
    linewidth=0.2,
    cmap="Blues",
    legend=True,
    legend_kwds={'shrink': 0.4},
)

In [7]:
ny_precinct_gdf = gpd.read_file("week6_NY_precincts")

# Practice Prorating Data

Now, let's practice prorating data.

Often we need to [`prorate`](https://maup.readthedocs.io/en/latest/user/prorating/) data when we have different geometries that do not overlap. This problem frequently occurs with election and redistricting data when we need to convert between *precincts* and *Census tracts.* We can do this easily using the fact that both precincts and Census tracts are comprised of *Census blocks.*

Consider the following problem. We have election results at the *precinct* level and we want to compare the results to demographic information at the *Census tract* level.

The basic steps of prorating data from *precincts* to *Census tracts* are as follows:

1. Collect your data at the precinct level
2. Assign your Census blocks to precincts
3. Determine the *population weights* for each Census block -- i.e., identify what proportion of the total *precinct* population is contained in each Census block that is contained within the precinct
4. Convert your data to the Census block level using `maup.prorate()`
5. Aggregate your data back up to the Census tract level -- typically, this step involves a `groupby()`
  * Use the fact that the first 11 digits of the Census block GEOID are the Census tract GEOID

We'll break this down!

## Step 1: Collect your data at the precinct level

In this case, our election outcomes are all at the precinct level.

In [10]:
# some data cleaning from Week6
sen_columns = [col for col in ny_precinct_gdf if col.startswith("USSen_")]
ny_precinct_gdf["USSen_total"] = ny_precinct_gdf[sen_columns].sum(axis=1)
sen_columns.append("USSen_total")

# look at our precinct level data
ny_precinct_gdf.head()

,GEOID,CountyFP,EDName,USSen_DEM,USSen_WOR,USSen_REP,USSen_CON,USSen_LAR,USSen_Writ,geometry,USSen_total
0,360010101001,001,Albany Ward 1 ED 1,39,6,9,0,0,0,"MULTIPOLYGON (((1038891.425 898237.779, 103888...",54
1,360010101010,001,Albany Ward 1 ED 10,7,0,0,0,0,0,"POLYGON ((1048486.377 900489.622, 1048847.807 ...",7
2,360010101002,001,Albany Ward 1 ED 2,146,30,21,6,1,0,"POLYGON ((1040970.913 900781.014, 1040656.324 ...",204
3,360010101003,001,Albany Ward 1 ED 3,182,21,47,5,0,2,"POLYGON ((1040894.602 900719.937, 1040999.140 ...",257
4,360010101004,001,Albany Ward 1 ED 4,172,22,28,4,2,0,"POLYGON ((1045046.865 900050.239, 1044979.039 ...",228


In [11]:
# there was an invalid geometry error in this shapefile, and this was a quick fix
county_precinct_gdf = ny_precinct_gdf[ny_precinct_gdf.CountyFP == county_fips]
county_precinct_gdf.geometry = county_precinct_gdf.geometry.buffer(0)

## Step 2. Assign Census blocks to precincts

In [12]:
# maup.assign will take each block and figure out what precinct it is in
block_to_precinct_assignment = maup.assign(county_block_gdf, county_precinct_gdf)
block_to_precinct_assignment = block_to_precinct_assignment[~pd.isna(block_to_precinct_assignment)].astype(int)
block_to_precinct_assignment

,0
GEOID20,
360470053020001,6066
360471058041002,9069
360470352000001,7831
360470348001014,9093
360470702031005,8530
...,...
360470960001011,8460
360471220002015,8535
360471124002001,8563


## Step 3. Produce the population weights

For each Census block, we calcualte what share of the precinct's population it contains. [`map`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.map.html) is a Pandas function that in this case will produce the total precinct population for every Census block assigned to that precinct.

In [20]:
pop_weights = (
    county_block_with_race_populated_gdf["total"]
    / block_to_precinct_assignment.map(county_block_with_race_populated_gdf["total"].groupby(block_to_precinct_assignment).sum())
).fillna(0)
pop_weights

,0
GEOID20,
360470001001000,0.011882
360470001001001,0.017110
360470001001002,0.000000
360470001001003,0.000000
360470001001004,0.006654
...,...
360471522003006,0.102273
360471522003007,0.057054
360471522003008,0.044939


## Step 4. Prorate!

Using `maup.prorate()` we can convert our precinct level data to the Census block level. `maup.prorate()` takes in three arguments:
1. The block to precinct assignment
2. The data we want to prorate (at the precinct level)
3. The population weights

In [22]:
# prorate! the resulting data is at the Census block level
aggregated_votes_df = maup.prorate(block_to_precinct_assignment, county_precinct_gdf[sen_columns], weights=pop_weights)
aggregated_votes_df

,USSen_DEM,USSen_WOR,USSen_REP,USSen_CON,USSen_LAR,USSen_Writ,USSen_total
GEOID20,,,,,,,
360470053020001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
360471058041002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
360470352000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
360470348001014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
360470702031005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
360470960001011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
360471220002015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
360471124002001,30.692199,1.495035,1.231206,0.263830,0.087943,0.000000,33.770213


## Step 5. Aggregate up to the Census tract level

We use the fact that the census tract is the first 11 characters of the Census block GEOID. We can then group the data by Census tract and sum. The resulting data has our election information at the *Census tract* level. We can now combine this with Census data from the American community survey to study meaningful correlations and trends.

In [23]:
# we use the fact that the census tract is the first 11 characters of the Census block GEOID
aggregated_votes_df["tract"] = aggregated_votes_df.index.str.slice(0, 11)

In [24]:
# now we can group the votes by the Census tract and sum
aggregated_tract_votes_df = aggregated_votes_df.groupby("tract")[sen_columns].sum()
aggregated_tract_votes_df.head()

,USSen_DEM,USSen_WOR,USSen_REP,USSen_CON,USSen_LAR,USSen_Writ,USSen_total
tract,,,,,,,
36047000100,1558.538829,235.336135,193.901335,9.608714,16.835505,1.755448,2015.975966
36047000200,94.475314,20.346497,24.658869,2.680281,2.103785,0.201089,144.465834
36047000301,1350.000408,224.601212,137.156974,12.057659,7.355531,4.662339,1735.834122
36047000501,1300.674395,204.873967,132.738919,16.264190,8.442001,6.183161,1669.176633
36047000502,1036.682356,192.875709,109.639555,10.454243,5.071409,2.750711,1357.473983
...,...,...,...,...,...,...,...
36047121400,475.443191,17.181305,32.908541,0.727236,2.181708,0.000000,528.441982
36047122000,856.156809,21.418695,53.891459,2.272764,3.818292,2.000000,939.558018
36047123700,695.000000,53.000000,483.000000,30.000000,4.000000,15.000000,1280.000000
